In [2]:
# import statements

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

2024-03-17 06:07:40.539591: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 06:07:41.627082: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 06:07:44.271992: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 06:07:47.454206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#defining a sample of names/data
character_names = ["gabriella", "ryan", "cecilia", "kari", "dimer", "allan", "sarah", "eliza", "Aaron", 
                "katnis", "henry", "ava", "alice", "bob", "john", "alberto", "marcos", "vex", "zach",
                "shabaaz", "pharoah", "mia", "katherine", "farrow", "ivy", "jack", "emily", "charlie", 
                "espoir", "patrick", "steph", "jeffrey", "diana", "peter", "pascal"]

#create a vocabulary based on the sample names
vocab = sorted(set(''.join(character_names)))
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

#Convert character names to sequences
sequences = [[char_to_idx[char] for char in name] for name in character_names]

#padding our sequences to ensure equal name length data
max_name_length = max(len(seq) for seq in sequences)
#print(max_name_length)
padded_sequences = pad_sequences(sequences, maxlen=max_name_length, padding='post')

In [4]:
#Training our model
#input and target sequences
x = padded_sequences[:, :-1] 
y = padded_sequences[:, 1:]

#check the shape of input + target sequence
print(x.shape)
print(y.shape)

(35, 8)
(35, 8)


In [5]:
#convert target sequence into a one-hot encoding
y_one_hot = to_categorical(y, num_classes=len(vocab))

#defining characteristics for our gru-based charcater generation model
vocab_size = len(vocab)
embedding_dim = 12


In [9]:
#creating the model
#run into problems with Embedding input_length=max_name_length
#return statement said that is not required/recognize it 
model = tf.keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    GRU(128, return_sequences=True),
    Dense(vocab_size, activation='softmax')
])

In [12]:
#Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy')
#training the model
model.fit(x, y_one_hot, epochs=50, verbose=1)

Epoch 1/50


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 3.2188
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1905 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1661 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1302 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.0893 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.0166 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.9596 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.8417 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.6840
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.4637 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1629 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.2312 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0210 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.0954 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0787 
Epoch 16/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/st

In [ ]:
#declare seed character 
seed = 'a'

#map our seed indexes to characters
seed_idx = [char_to_idx[char] for char in seed]

#actaully calling our model
#finding the probability of the model
#generating a name :
for _ in range(3):
    temperature = .5
    predicted_probs = model.predict(np.array(seed_idx).reshape(1, -1))[0]
    predicted_probs = predicted_probs[-1]